In [0]:
from delta.tables import DeltaTable

def upsert_into_delta_table(upd_df, table_path,merge_condition,partition_col = None):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning","true")

    if spark.catalog.tableExists(table_path):

        resultsDeltaTable = DeltaTable.forName(spark, table_path)

        resultsDeltaTable.alias('tgt') \
        .merge(
                upd_df.alias('upd'), 
                merge_condition) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    else:
        if partition_col:
            writer = upd_df.write \
                .format("delta") \
                .mode("overwrite") \
                .partitionBy(partition_col)
        else:
            writer = upd_df.write \
                .format("delta") \
                .mode("overwrite")

        writer.saveAsTable(table_path)

In [0]:
def push_partition_col_to_end(df, partition_col):
    cols = df.schema.names
    cols.remove(partition_col)
    cols.append(partition_col)
    return df.select(cols)

In [0]:
def merge_delta_data(input_df, db_name, table_name, merge_condition, partition_column):
  # spark.conf.set("spark.databricks.optimizer.dynamicFilePruning","true")

  from delta.tables import DeltaTable
  if (spark.catalog.tableExists(f"{db_name}.{table_name}")):
    deltaTable = DeltaTable.forName(spark, f"{db_name}.{table_name}")
    deltaTable.alias("tgt").merge(
        input_df.alias("src"),
        merge_condition) \
      .whenMatchedUpdateAll()\
      .whenNotMatchedInsertAll()\
      .execute()
  else:
    input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{db_name}.{table_name}")

In [0]:
def df_column_to_list(input_df, column_name):
  df_row_list = input_df.select(column_name) \
                        .distinct() \
                        .collect()
  
  column_value_list = [row[column_name] for row in df_row_list]
  return column_value_list